In [1]:
pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.2 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ete3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 12.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ete3: filename=ete3-3.1.3-py3-none-any.whl size=2273785 sha256=b20eb9fff84d8c0d4cd8a5adcec1a087ae189df43952f46ab3ed6f401cfa3753
  Stored in directory: /Users/user/Library/Caches/pip/wheels/ad/2e/cc/edcca721b423e1604c84f480a1e8e0547a223bfc068d373259
Successfully built ete3
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ete3 import Tree

# Create a simple unrooted binary tree
newick_tree = "(human, (chimp, (mouse, rat)));"
tree = Tree(newick_tree)

# Visualize the tree
print(tree)

# Traverse the tree and print its structure
for node in tree.traverse("postorder"):
    print(f"Node: {node.name}, Is leaf: {node.is_leaf()}")

# Collapse a common cherry (e.g., chimp and human)
tree.get_common_ancestor("chimp", "human").delete()

print("After collapsing common ancestor of chimp and human:")
print(tree)



   /-human
--|
  |   /-chimp
   \-|
     |   /-mouse
      \-|
         \-rat
Node: human, Is leaf: True
Node: chimp, Is leaf: True
Node: mouse, Is leaf: True
Node: rat, Is leaf: True
Node: , Is leaf: False
Node: , Is leaf: False
Node: , Is leaf: False
After collapsing common ancestor of chimp and human:

   /-human
--|
  |   /-chimp
   \-|
     |   /-mouse
      \-|
         \-rat


In [5]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a phylogenetic tree using an undirected graph
tree = nx.Graph()

# Add nodes (species) and edges (relationships)
tree.add_edges_from([
    ("root", "A"), ("A", "B"), ("A", "C"),
    ("C", "D"), ("C", "E")
])

# Visualize the tree
pos = nx.spring_layout(tree)
nx.draw(tree, pos, with_labels=True, node_size=500, node_color='lightblue')
plt.show()


TypeError: '_AxesStack' object is not callable

<Figure size 640x480 with 0 Axes>

In [7]:
def compute_maf(tree1, tree2):
    """
    Basic conceptual approach for MAF simulation
    :param tree1: Phylogenetic tree 1 (using NetworkX or ETE)
    :param tree2: Phylogenetic tree 2
    :return: Number of components in the Maximum Agreement Forest
    """
    # Step 1: Identify cherries in both trees
    def find_cherries(tree):
        cherries = []
        for node in tree.nodes:
            neighbors = list(tree.neighbors(node))
            if len(neighbors) == 2 and all(tree.degree(n) == 1 for n in neighbors):
                cherries.append(neighbors)
        return cherries

    cherries1 = find_cherries(tree1)
    cherries2 = find_cherries(tree2)

    # Step 2: Collapse common cherries
    common_cherries = set(tuple(sorted(c)) for c in cherries1) & set(tuple(sorted(c)) for c in cherries2)
    print(f"Common cherries: {common_cherries}")

    # Step 3: Count remaining components as MAF size
    remaining_components = len(tree1.nodes) - len(common_cherries)
    return remaining_components


# Example simulation using NetworkX trees
tree1 = nx.Graph()
tree1.add_edges_from([("A", "B"), ("B", "C"), ("B", "D")])
tree2 = nx.Graph()
tree2.add_edges_from([("A", "B"), ("B", "C"), ("B", "E")])

maf_size = compute_maf(tree1, tree2)
print(f"Computed MAF size: {maf_size}")


Common cherries: set()
Computed MAF size: 4


In [8]:
from Bio import Phylo
from io import StringIO

# Define a simple rooted Newick tree
tree_data = "((A,B),(C,D));"  # This is a simple rooted tree with cherries {A, B} and {C, D}
tree = Phylo.read(StringIO(tree_data), "newick")

# Visualize the tree structure
Phylo.draw_ascii(tree)

# Access nodes and check for cherries
for clade in tree.get_nonterminals():
    children = clade.clades
    if len(children) == 2 and all(child.is_terminal() for child in children):
        print(f"Cherry found: {children[0].name}, {children[1].name}")


                                        _____________________________________ A
  _____________________________________|
 |                                     |_____________________________________ B
_|
 |                                      _____________________________________ C
 |_____________________________________|
                                       |_____________________________________ D

Cherry found: A, B
Cherry found: C, D


In [9]:
import networkx as nx

def widden_branch(tree_graph, cherry):
    """
    Simulate Whidden's branching for a cherry in a rooted tree.
    :param tree_graph: Graph representation of a rooted tree
    :param cherry: Tuple (a, b) representing a cherry
    """
    a, b = cherry
    path_edges = list(nx.shortest_path(tree_graph, a, b))[:-1]
    print(f"Path between {a} and {b}: {path_edges}")

    # Three branching choices
    print("Branch 1: Cut off a")
    print("Branch 2: Cut off b")
    print(f"Branch 3: Cut all but one of the path edges ({path_edges[1:]})")

# Example tree as a graph
tree_graph = nx.DiGraph()
tree_graph.add_edges_from([
    ("root", "A"), ("root", "B"),
    ("root", "C"), ("C", "D"), ("C", "E")
])

# Test with a cherry
widden_branch(tree_graph, ("A", "B"))


NetworkXNoPath: No path between A and B.